# 원티드 프리온보딩 선발과제

# 문제 1) Tokenizer 생성하기

In [53]:
import numpy as np
from math import log as ln
import re

class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False
  
    def preprocessing(self, sequences):
        result=[]
        '''
        문제 1-1
        '''
        for sequence in sequences:
            sentence=re.sub(r"[^a-zA-Z0-9]", " ", sequence.lower()) #특수문자 제거 및 소문자 처리
            result.append(sentence.split()) #white space
        return result
    
    def fit(self, sequences):
        self.fit_checker = False
        '''
        문제 1-2
        '''
        #토큰화
        tokens = self.preprocessing(sequences)
        word_dict = self.word_dict
        
        #word_dict
        for token in tokens:
            for word in token:
                if word not in list(word_dict.keys()):
                    word_dict[word]= len(word_dict)
        self.word_dict = word_dict
        self.fit_checker = True

    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)
        if self.fit_checker:
            '''
            문제 1-3
            '''
            
            #정수 인덱싱
            for token in tokens:
                word_idx=[]
                for word in token:
                    if word not in list(self.word_dict.keys()):
                        word_idx.append(self.word_dict['oov'])
                    else:
                        word_idx.append(self.word_dict[word])
                result.append(word_idx)
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")

    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result


In [56]:
token=Tokenizer()
token.preprocessing(sequences)

[['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]

# 문제 2) TfidfVectorizer 생성하기

In [54]:
#python
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False

    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        '''
        문제 2-1번
        '''
        #idf
        total_word_idx= np.unique(np.concatenate(tokenized).tolist())
        N=len(tokenized) #문장 개수
        idf=[]
        
        for idx in total_word_idx:
            df=0
            for token in tokenized:
                if idx in token:
                    df+=1
            idf.append(ln(N/(df+1))) #자연로그
        
        self.idf=idf
        self.total_word_idx=total_word_idx
        self.fit_checker = True

    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            '''
            문제 2-2.
            '''            
            #tf
            tf=[]
            for token in tokenized:
                token_tf=[]
                for i in self.total_word_idx:
                    token_tf.append(token.count(i))
                tf.append(token_tf)
                
            #broadcasting
            self.tfidf_matrix= (np.array(tf)*np.array([self.idf])).tolist() #nest list

            return self.tfidf_matrix

        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")
            
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [50]:
tfidf=TfidfVectorizer(Tokenizer())
tfidf.fit(sequences)
tfidf.fit_transform(sequences)

[[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]]